In [ ]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.sql('''select 'spark' as hello ''')
df.show()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local[2]").appName("word
count").getOrCreate()

The next cell reads a dataset and uses coalesce to minimize the data shuffled and filter out the records where description is null.
read.json results in a dataframe where spark can understand the schema of the json meaning he can get each row and understande it. whereas read.praquet converts all column to be nullable

In [ ]:
sales = spark.read.format("csv")\
 .option("header", "true")\
 .option("inferSchema", "true")\
 .load("./data/retail-data/by-day/*.csv")\
 .coalesce(5)\
 .where("Description IS NOT NULL")
simpleDF = spark.read.json("/.data/simple-ml")
scaleDF = spark.read.parquet("./data/simple-ml-scaling")

standard scaler in general transforms the dataset, each column to have a standard deviation of 1 and 0-mean. our sclaer has input column feature. the scaler will compute the results on scaleDF (this step generates the model ) then his model is used to transform the scaleDF to have a standard deviation of 1. 

In [ ]:
from pyspark.ml.feature import StandardScaler
sScaler = StandardScaler().setInputCol("features")
sScaler.fit(scaleDF).transform(scaleDF).show()

min max scaler transform features to be in a specific range in this case from 5 to 10(same as above) the model is called here fittedminMax it has minimum vlue 5 and maximum 10 and then is used to transform scaleDF to be in the specified range

In [ ]:
from pyspark.ml.feature import MinMaxScaler
minMax = MinMaxScaler().setMin(5).setMax(10).setInputCol("features")
fittedminMax = minMax.fit(scaleDF)
fittedminMax.transform(scaleDF).show()

the normalizer normalizes the features to have a unit form. Here the p-norm is 1. manhattanDistance is our model and it is used to fit scaleDF. 

In [ ]:
from pyspark.ml.feature import Normalizer
manhattanDistance = Normalizer().setP(1).setInputCol("features")
manhattanDistance.transform(scaleDF).show()

what i understand is that stringIndexer counts the number of occurences of a certain value and sorts them with respect to which came more. the more the occurences the less the index. meaning index 0 is the most occurence index 1 occured less and so on

In [ ]:
from pyspark.ml.feature import StringIndexer
lblIndxr = StringIndexer().setInputCol("lab").setOutputCol("labelInd")
idxRes = lblIndxr.fit(simpleDF).transform(simpleDF)
idxRes.show()

it is not best practice to reuse a stringIndexer since the default is to throw an error

In [ ]:
valIndexer = StringIndexer().setInputCol("value1").setOutputCol("valueInd")
valIndexer.fit(simpleDF).transform(simpleDF).show()

here we use the indexer to string index the color so now colorInd is not string it is indices then we use one hot encoder to use colorind to make it a column of binary vectors(normal one hot encoding) 

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
lblIndxr = StringIndexer().setInputCol("color").setOutputCol("colorInd")
colorLab = lblIndxr.fit(simpleDF).transform(simpleDF.select("color"))
ohe = OneHotEncoder().setInputCol("colorInd")
ohe.transform(colorLab).show()

this breaks sentences (same like the usual split in java forexample) on the pattern (" ") from Description to Dexcout and sets all words to lowercase

In [ ]:
from pyspark.ml.feature import RegexTokenizer
rt = RegexTokenizer()\
 .setInputCol("Description")\
 .setOutputCol("DescOut")\
 .setPattern(" ")\
 .setToLowercase(True)
rt.transform(sales.select("Description")).show(20, False)

this uses the tokenizer above. chiSqSelector from what i understand is that it uses categorical data and uses the chi-squared test to test if there were dependant or not.

In [ ]:
from pyspark.ml.feature import ChiSqSelector, Tokenizer
tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn\
 .transform(sales.select("Description", "CustomerId"))\
 .where("CustomerId IS NOT NULL")
prechi = fittedCV.transform(tokenized)\
 .where("CustomerId IS NOT NULL")
chisq = ChiSqSelector()\
 .setFeaturesCol("countVec")\
 .setLabelCol("CustomerId")\
 .setNumTopFeatures(2)
chisq.fit(prechi).transform(prechi)\
 .drop("customerId", "Description", "DescOut").show()